In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from joblib import load
from tqdm import tqdm

from utils import create_data_loaders, train_model, test_model

#### Hyperparameters

In [2]:
RANDOM_STATE = 42
BATCH_SIZE = 32
NUM_EPOCHS = 40
LEARNING_RATE = 1e-3
PATIENCE = 10
MIN_DELTA = 1e-4

### Baseline-1: LSTM (hidden) + ff

In [3]:
class DualLSTMModel(nn.Module):
    def __init__(self,
                 run_size = 20,
                 incoming_run_size = 45,
                 run_hidden_size = 128,
                 incoming_run_hidden_size = 128,
                 num_layers = 1,
                 dropout = 0.2,
                 ff_hidden_sizes=None,
                 ff_output_size=49):
        super().__init__()
        self.run_size = run_size
        self.incoming_run_size = incoming_run_size
        self.run_hidden_size = run_hidden_size
        self.incoming_run_hidden_size = incoming_run_hidden_size

        if ff_hidden_sizes is None:
            ff_hidden_sizes = [128, 64]
        self.lstm_run = nn.LSTM(
            input_size=run_size,
            hidden_size=run_hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True,
        )

        self.lstm_incoming_run = nn.LSTM(
            input_size=incoming_run_size,
            hidden_size=incoming_run_hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )

        last_output_size = run_hidden_size + incoming_run_hidden_size
        ff_layers = []
        prev_hidden_size = last_output_size

        for hidden_size in ff_hidden_sizes:
            ff_layers.extend([
                nn.Linear(prev_hidden_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_hidden_size = hidden_size

        ff_layers.append(nn.Linear(prev_hidden_size, ff_output_size))
        self.fead_forward = nn.Sequential(*ff_layers)

    def forward(self, x1, x2, lengths1, lengths2):
        if lengths1 is not None:
            x1_packed = nn.utils.rnn.pack_padded_sequence(
                x1, lengths1.cpu(), batch_first=True, enforce_sorted=False
            )
            lstm1_out_packed, (h1_n, c1_n) = self.lstm_run(x1_packed)
            out_run = h1_n[-1]
        else:
            lstm1_out, (h1_n, c1_n) = self.lstm_run(x1)
            out_run = h1_n[-1]

        if lengths2 is not None:
            x2_packed = nn.utils.rnn.pack_padded_sequence(
                x2, lengths2.cpu(), batch_first=True, enforce_sorted=False
            )
            lstm2_out_packed, (h2_n, c2_n) = self.lstm_incoming_run(x2_packed)
            out_incoming_run = h2_n[-1]
        else:
            lstm2_out, (h2_n, c2_n) = self.lstm_incoming_run(x2)
            out_incoming_run = h2_n[-1]

        return self.fead_forward(torch.concat([out_run, out_incoming_run], dim=1))

In [4]:
# Model summary
from torchinfo import summary

model = DualLSTMModel()

summary(
    model,
    input_data=(
        torch.randn(32, 755, 20),  # x1
        torch.randn(32, 755, 45),  # x2
        torch.full((32,), 700),    # lengths1
        torch.full((32,), 700)     # lengths2
    )
)

Layer (type:depth-idx)                   Output Shape              Param #
DualLSTMModel                            [32, 49]                  --
├─LSTM: 1-1                              [22400, 128]              76,800
├─LSTM: 1-2                              [22400, 128]              89,600
├─Sequential: 1-3                        [32, 49]                  --
│    └─Linear: 2-1                       [32, 128]                 32,896
│    └─ReLU: 2-2                         [32, 128]                 --
│    └─Dropout: 2-3                      [32, 128]                 --
│    └─Linear: 2-4                       [32, 64]                  8,256
│    └─ReLU: 2-5                         [32, 64]                  --
│    └─Dropout: 2-6                      [32, 64]                  --
│    └─Linear: 2-7                       [32, 49]                  3,185
Total params: 210,737
Trainable params: 210,737
Non-trainable params: 0
Total mult-adds (Units.GIGABYTES): 477.10
Input size (MB): 6.28
F

In [4]:
run_matrices = load('./data/processed/run_matrices.joblib')
incoming_run_matrices = load('./data/processed/incoming_run_matrices.joblib')
metrology_matrix = load('./data/processed/metrology_matrix.joblib')

X_run = torch.from_numpy(run_matrices).float()
X_incoming_run = torch.from_numpy(incoming_run_matrices).float()
y = torch.from_numpy(metrology_matrix).float()
print(X_run.shape, X_incoming_run.shape, y.shape)

torch.Size([4140, 755, 20]) torch.Size([4140, 755, 45]) torch.Size([4140, 49])


In [5]:
baseline_1_model = DualLSTMModel(
    run_hidden_size=128,
    incoming_run_hidden_size=128,
    num_layers=1,
    dropout=0.2,
    ff_hidden_sizes=[128, 64]
)

train_loader, val_loader, test_loader = create_data_loaders(X_run, X_incoming_run, y, train_ratio=0.7, val_ratio=0.1, batch_size=BATCH_SIZE, standardize=True, random_state=RANDOM_STATE)

train_losses, val_losses = train_model(baseline_1_model, train_loader, val_loader, num_epochs=1, learning_rate=LEARNING_RATE, patience=PATIENCE, min_delta=MIN_DELTA, model_save_path='baseline-1-best-model.pth')

expanded_mask: torch.Size([4140, 755, 1])
standardized: torch.Size([4140, 755, 20])
expanded_mask: torch.Size([4140, 755, 1])
standardized: torch.Size([4140, 755, 45])
Epoch 1/1
Train Loss: 23.877116, Val Loss: 0.081266
Learning Rate: 1.00e-03
--------------------


/opt/anaconda3/envs/nlp_2025/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
test_results = test_model(baseline_1_model, test_loader)

Testing model on 26 batches...


In [7]:
print({k: test_results[k] for k in ['test_loss', 'mse', 'mae', 'r2_score']})

{'test_loss': 0.08184193920057553, 'mse': 0.08184580504894257, 'mae': 0.21791943907737732, 'r2_score': -1.7126636505126953}
